In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.1 MB/s eta 0:00:00


In [3]:
# ===================================================================
#  Library
# ===================================================================
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.simplefilter("ignore")

import numpy as np
from scipy.optimize import minimize
import optuna
import lightgbm as lgb

In [4]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    n_seeds = 3
    n_trials = 2000
    save_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/exp/"
    data_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/"
    filename = "exp00062"

In [5]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [6]:
kun_exp00055_oof_pred= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00055_oof_pred.csv').rename(columns={'optimized_pred': 'kun_pred_0'})
kun_exp00056_oof_pred = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00056_oof_pred.csv').rename(columns={'oof_pred': 'kun_pred_1'})
kun_oof =  kun_exp00055_oof_pred.merge(kun_exp00056_oof_pred, on='id')


oof_df_exp051 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp051.csv').rename(columns={'pred': 'pred_0'})
oof_df_exp052 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp052.csv').rename(columns={'pred': 'pred_1'})
oof_df_exp053 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp053.csv').rename(columns={'pred': 'pred_2'})
oof_df_exp054 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp054.csv').rename(columns={'pred': 'pred_3'})
yuji_oof = pd.concat([oof_df_exp051, oof_df_exp052['pred_1'], oof_df_exp053['pred_2'], oof_df_exp054['pred_3']],axis=1)


kun_exp00055 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00055.csv', header=None).rename(columns={0:'id', 1: 'kun_pred_0'})
kun_exp00056 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00056.csv', header=None).rename(columns={0:'id', 1: 'kun_pred_1'})
kun_test =  kun_exp00055.merge(kun_exp00056, on='id')


exp051 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp051.csv', header=None).rename(columns={0:'id', 1: 'pred_0'})
exp052 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp052.csv', header=None).rename(columns={0:'id', 1: 'pred_1'})
exp053 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp053.csv', header=None).rename(columns={0:'id', 1: 'pred_2'})
exp054 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp054.csv', header=None).rename(columns={0:'id', 1: 'pred_3'})
yuji_test = pd.concat([exp051, exp052['pred_1'], exp053['pred_2'], exp054['pred_3']],axis=1)


In [7]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')
df = pd.concat([kun_oof.merge(yuji_oof, on='id'), df_train['price']], axis=1)
test = kun_test.merge(yuji_test, on='id')

In [8]:
df

,id,kun_pred_0,kun_pred_1,pred_0,pred_1,pred_2,pred_3,price
0,0,8805.219623,7895.4160,8759.290471,8929.546032,8916.322165,8977.757097,27587
1,1,3799.844508,4177.3580,3859.910547,3825.508199,3822.104706,3876.047282,4724
2,2,2959.861854,2818.1372,3027.760649,3049.493176,2982.516292,3056.648628,10931
3,3,7934.654500,8179.8460,7990.517289,8005.600906,7993.295606,7894.338559,16553
4,4,4461.927715,4321.6885,4524.287997,4479.006617,4569.630683,4570.899182,5158
...,...,...,...,...,...,...,...,...
27527,27527,12385.902685,12967.0370,12813.569454,12748.693982,12889.434904,13184.273766,32212
27528,27528,6939.817355,6854.7417,6735.046172,6676.772957,6772.174105,6772.583423,5400
27529,27529,14186.293321,15186.2150,14571.234351,14813.291564,14599.654721,14877.525968,22227
27530,27530,6658.054764,6914.8745,6651.976275,6762.372206,6798.175222,6822.692681,3054


In [9]:
test

,id,kun_pred_0,kun_pred_1,pred_0,pred_1,pred_2,pred_3
0,27532,9347.528622,8806.765109,9681.016732,9631.179775,9535.601879,9589.502294
1,27533,5473.589480,5556.561678,5589.905122,5625.972274,5579.913722,5696.370941
2,27534,5542.547482,5788.564407,5588.496733,5559.707831,5556.921123,5567.494508
3,27535,17815.695204,19133.043389,18539.014190,18585.315662,18690.825713,18540.038374
4,27536,4446.383701,4014.969884,4461.198006,4460.135571,4474.330698,4508.779351
...,...,...,...,...,...,...,...
27532,55064,12602.958879,14460.748909,13135.222052,12961.122998,12965.397426,13145.010785
27533,55065,9228.375920,8297.307991,9473.916681,9638.115991,9440.222225,9526.790475
27534,55066,5535.535863,5972.723292,5564.120273,5513.388636,5554.669582,5468.090477
27535,55067,4766.881828,4963.743941,4795.089266,4829.818804,4832.357339,4831.827553


In [10]:
def get_custom_cv(df, n_splits=5):
    # priceを小さい順に各foldに振り分ける
    df = df.sort_values(by="price", ignore_index=True)
    df["fold"] = [i for i in range(n_splits)] * (df.shape[0] // n_splits) \
                + [i for i in range(df.shape[0] % n_splits)]
    df = df.sort_values(by="id", ignore_index=True)


    for fold in range(n_splits):
        train_idx = df[df["fold"] != fold].index
        valid_idx = df[df["fold"] == fold].index
        yield train_idx, valid_idx

n_splits = 8
cv = list(get_custom_cv(df, n_splits))

In [11]:
def fit_lgbm(X, y, test_data, cv, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)
    test_list = []

    for i, (idx_train, idx_valid) in enumerate(cv):
        X_tr, y_tr = X.iloc[idx_train], y.iloc[idx_train]
        X_val, y_val = X.iloc[idx_valid], y.iloc[idx_valid]
        transformed_test_data = test_data.copy()


        drop_cols = ['id', 'price']
        X_tr.drop(drop_cols, axis=1, inplace=True)
        X_val.drop(drop_cols, axis=1, inplace=True)
        transformed_test_data.drop(['id'], axis=1, inplace=True)
        test_list.append(transformed_test_data)


        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr,
            y_tr,
            categorical_feature=categorical_cols,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(100, verbose=verbose)],
        )
        pred_i = model.predict(X_val)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = mean_absolute_percentage_error(y_val, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(y, oof_pred)
    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models, test_list




# optunaを使ったパラメータチューニング
def tuning(df, cv):
     def objective(trial):
         max_depth = trial.suggest_int("max_depth", 1, 10)
         params = {
              "max_depth": max_depth,
              "n_estimators": trial.suggest_int('n_estimators', 100, 50000),
              "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
              "num_leaves": trial.suggest_int("num_leaves", 2, 2**max_depth),
              "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1.0),
              "subsample": trial.suggest_float("subsample", 0.01, 1.0),
              "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 0, 300),
              "max_bin": trial.suggest_int("max_bin", 32, 512),
              "reg_lambda": trial.suggest_float("reg_lambda", 1e-10, 100.0),
              "task":"train",
              "objective": "mape",
              "boosting":"gbdt",
              "metric": 'mape',
              "importance_type": "gain",
              "seed" : 88,
              "verbosity": -1,


         }

         score, _, _, _ = fit_lgbm(
             X=df, y=df["price"], test_data=test, cv=cv,  categorical_cols=[], params=params, verbose=-1
         )
         return score

     study = optuna.create_study(direction="minimize")
     study.optimize(objective, n_trials=100)
     print("Number of finished trials:", len(study.trials))
     print("Best trial:", study.best_trial.params)
     return study.best_trial.params

In [12]:
lgbm_params = tuning(df, cv)

lgbm_params.update(
    {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "importance_type": "gain",
    "seed" : 88,
    "verbosity": -1,
    }
)


# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    X=df,
    y=df["price"],
    test_data=test,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

[I 2023-08-22 07:13:28,270] A new study created in memory with name: no-name-b9dda5d5-c7b1-4922-a588-4d09a1ac985e


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5]	valid_0's mape: 0.434549
 - fold1 - 0.4345
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[7]	valid_0's mape: 0.443028
 - fold2 - 0.4430
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's mape: 0.434069
 - fold3 - 0.4341
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.446341
 - fold4 - 0.4463
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's mape: 0.428332
 - fold5 - 0.4283
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5]	valid_0's mape: 0.430931
 - fold6 - 0.4309
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's mape: 0.445336
 - fold7 -

[I 2023-08-22 07:13:35,992] Trial 0 finished with value: 0.4373102146413268 and parameters: {'max_depth': 9, 'n_estimators': 25628, 'learning_rate': 0.50173412497315, 'num_leaves': 52, 'colsample_bytree': 0.8757797118266991, 'subsample': 0.38354719789585223, 'min_data_in_leaf': 267, 'max_bin': 117, 'reg_lambda': 31.96597896245935}. Best is trial 0 with value: 0.4373102146413268.


Early stopping, best iteration is:
[7]	valid_0's mape: 0.435894
 - fold8 - 0.4359
FINISHI: Whole Score: 0.4373
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.435752
 - fold1 - 0.4358
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.444955
 - fold2 - 0.4450
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.434123
 - fold3 - 0.4341
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's mape: 0.446589
 - fold4 - 0.4466
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	valid_0's mape: 0.432132
 - fold5 - 0.4321
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5]	valid_0's mape: 0.431658
 - fold6 - 0.4317
Training until validation

[I 2023-08-22 07:13:42,679] Trial 1 finished with value: 0.43863843659442353 and parameters: {'max_depth': 9, 'n_estimators': 26033, 'learning_rate': 0.6666027459388968, 'num_leaves': 391, 'colsample_bytree': 0.9313680790096143, 'subsample': 0.3016653279411466, 'min_data_in_leaf': 234, 'max_bin': 420, 'reg_lambda': 72.1945749920794}. Best is trial 0 with value: 0.4373102146413268.


...

Early stopping, best iteration is:
[174]	valid_0's mape: 0.434296
 - fold8 - 0.4343
FINISHI: Whole Score: 0.4358
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[89]	valid_0's mape: 0.43127
 - fold1 - 0.4313
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[89]	valid_0's mape: 0.44255
 - fold2 - 0.4426
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[157]	valid_0's mape: 0.431766
 - fold3 - 0.4318
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[98]	valid_0's mape: 0.444295
 - fold4 - 0.4443
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's mape: 0.429634
 - fold5 - 0.4296
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[58]	valid_0's mape: 0.427261
 - fold6 - 0.4273
Training until val

[I 2023-08-22 07:41:36,614] Trial 98 finished with value: 0.43591915442267204 and parameters: {'max_depth': 4, 'n_estimators': 33230, 'learning_rate': 0.08385629402404446, 'num_leaves': 6, 'colsample_bytree': 0.5620916963915259, 'subsample': 0.5462430027916453, 'min_data_in_leaf': 300, 'max_bin': 78, 'reg_lambda': 67.780472884598}. Best is trial 54 with value: 0.4353581859758386.


Early stopping, best iteration is:
[161]	valid_0's mape: 0.435822
 - fold8 - 0.4358
FINISHI: Whole Score: 0.4359
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[40]	valid_0's mape: 0.431726
 - fold1 - 0.4317
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[55]	valid_0's mape: 0.443071
 - fold2 - 0.4431
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[66]	valid_0's mape: 0.432002
 - fold3 - 0.4320
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's mape: 0.443815
 - fold4 - 0.4438
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[73]	valid_0's mape: 0.428818
 - fold5 - 0.4288
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[48]	valid_0's mape: 0.426762
 - fold6 - 0.4268
Training until va

[I 2023-08-22 07:41:46,825] Trial 99 finished with value: 0.43572005108038175 and parameters: {'max_depth': 4, 'n_estimators': 45195, 'learning_rate': 0.12263598609738144, 'num_leaves': 5, 'colsample_bytree': 0.5962456543731192, 'subsample': 0.592472015833028, 'min_data_in_leaf': 229, 'max_bin': 369, 'reg_lambda': 70.68664692915642}. Best is trial 54 with value: 0.4353581859758386.


Early stopping, best iteration is:
[127]	valid_0's mape: 0.435181
 - fold8 - 0.4352
FINISHI: Whole Score: 0.4357
Number of finished trials: 100
Best trial: {'max_depth': 4, 'n_estimators': 40964, 'learning_rate': 0.15375339480401906, 'num_leaves': 5, 'colsample_bytree': 0.5351032314077604, 'subsample': 0.5953542047799036, 'min_data_in_leaf': 215, 'max_bin': 50, 'reg_lambda': 60.40140902097511}
 - fold1 - 0.4312
 - fold2 - 0.4423
 - fold3 - 0.4309
 - fold4 - 0.4446
 - fold5 - 0.4278
 - fold6 - 0.4269
 - fold7 - 0.4440
 - fold8 - 0.4351
FINISHI: Whole Score: 0.4354


In [14]:
lgbm_params = {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "seed" : 88,
    "importance_type": "gain",
    "random_state": 88,
    "verbose": -1,
    'max_depth': 4,
    'n_estimators': 40964,
    'learning_rate': 0.15375339480401906,
    'num_leaves': 5,
    'colsample_bytree': 0.5351032314077604,
    'subsample': 0.5953542047799036,
    'min_data_in_leaf': 215,
    'max_bin': 50,
    'reg_lambda': 60.40140902097511
}


# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    X=df,
    y=df["price"],
    test_data=test,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

 - fold1 - 0.4312
 - fold2 - 0.4423
 - fold3 - 0.4309
 - fold4 - 0.4446
 - fold5 - 0.4278
 - fold6 - 0.4269
 - fold7 - 0.4440
 - fold8 - 0.4351
FINISHI: Whole Score: 0.4354


In [ ]:
Best trial: {'max_depth': 4, 'n_estimators': 40964, 'learning_rate': 0.15375339480401906, 'num_leaves': 5, 'colsample_bytree': 0.5351032314077604, 'subsample': 0.5953542047799036, 'min_data_in_leaf': 215, 'max_bin': 50, 'reg_lambda': 60.40140902097511}
 - fold1 - 0.4312
 - fold2 - 0.4423
 - fold3 - 0.4309
 - fold4 - 0.4446
 - fold5 - 0.4278
 - fold6 - 0.4269
 - fold7 - 0.4440
 - fold8 - 0.4351
==================================================
FINISHI: Whole Score: 0.4354

In [15]:
sub_df = pd.read_csv(CFG.data_dir + "submit_sample.csv", names=["id", "price"])

In [16]:
preds = []
for transformed_test_data, model in zip(transformed_test_data_list, models_lgb):
    pred = model.predict(transformed_test_data)
    preds.append(pred)
pred = np.mean(preds, axis=0)

# submission
sub_df["price"] = pred
sub_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{CFG.filename}.csv', index=False, header=False)

In [17]:
pd.concat([df['id'], pd.DataFrame(oof_lgb, columns=['oof_pred'])], axis=1).to_csv(f'/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_{CFG.filename}_oof_pred.csv', index=False)